## **Diffusion Equation**

$$
\frac{d\Sigma}{dt} = \frac{3}{r}\frac{\partial}{\partial r} \left[r^{1/2} \frac{\partial}{\partial r}(\nu \Sigma r^{1/2})\right]
$$

This is evolution equation for the surface density of a geometrically thin disk

Putting this in the form of a diffusion equation:

$$
\frac{\partial f}{\partial t} = D \frac{\partial^2 f}{\partial X^2}
$$
where $X \equiv 2r^{1/2}$, $f \equiv \frac{3}{2}\Sigma X$ and $D = \frac{12\nu}{X^2}$


We need numerical values for $\nu$
$$
\nu = \alpha c_s H \\
H = \frac{c_s}{\Omega} \\
\Omega = \sqrt{\frac{G*M_s}{r^3}} \\
c_s = \sqrt{\frac{k_B * T}{\mu * mH}} \\
mH = \frac{1}{N_A} \\
\mu = 2.3 \\
$$

The Temperature profile is given by:
$$
2 \sigma_b T_{disk}^4 = \frac{9}{4}\Sigma \nu \Omega^2\\
T^3 = \frac{9}{8\sigma_b}\Sigma \frac{k_b \alpha}{\mu mH}\sqrt{\frac{GM}{r^3}}
$$

The Pressure is given by:

$$
P = \frac{c_s^2\Sigma}{H} = c_s\Sigma\Omega
$$


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import pandas as pd
import os
from dust_diffuse import *
from thermal import *

In [2]:
#Natural Constants

# T = 600 # Temperature
mu = 2.2 # mean molar mass in disk
avog = 6.02214 * 10**23 # avogadros number
mH = 1.673534 * 10**-27 # atomic hydrogen mass
kB = 1.380649 * 10**-23 #boltzmann constant
G = 6.6738 * 10**-11 #gravitational constant
Ms = 1.9886 * 10**30 #solar mass
Me = 5.972 * 10 **24 #earth mass
AU = 1.4959787 * 10**11 #astronomical units
yrs2sec = 3.1536 * 10**7 #convert years to seconds
sb = 5.6704*10**-8

In [3]:
#Disk Parameters
alpha1 = 0.04
alpha2 = 1.0 * 10**-3
alpha3 = 1.0 * 10**-4
rin = 0.05 * AU #inner radius of the disk
rout = 30 * AU #outer radius of the disk
sigma_in  = 2 * 10**5 #boundary condition at t=0
sigma_max = sigma_in*2 #boundary condition at t=final_time
sigma_min = 1 * 10**2
distance = rout - rin #distance from inner radius to outer radius
St = .01 #Stokes Number
v_gas = 0

In [4]:
#Temporal discretization
max_years = 100
dyr = .01
dt = dyr * yrs2sec #timestep
final_time = max_years*yrs2sec + 1 #total diffusion time in seconds
t_save_interval = 10 #every ten years

In [5]:
#Spacial discretization

n = 300 #number of steps / number of points to plot
dr = distance/n #distance between each spacial step
cont = True

In [6]:
#Plotting axes
t = np.arange(0, final_time, dt)
# t_plot = []

In [7]:
#Initialize grid spaces

dist = np.empty(n)
Omega = np.empty(n)
nu = np.empty(n)
X = np.empty(n)
D = np.empty(n)
sigma_gas = np.empty(n)
f = np.empty(n)
sigma_dust = np.empty(n)


In [8]:
#Initialize Staggered Grid

dist_stag = np.empty(n-1)
Omega_stag = np.empty(n-1)
dPdr = np.empty(n-1)
cs2_stag = np.empty(n-1)
press_stag = np.empty(n-1)
rho_stag = np.empty(n-1)
v_dust = np.empty(n)

In [9]:
#Values that are saved

# sigma_gas_evol = []
# temp_evol = []
# press_evol = []
# mass_evol = []
# v_dust_evol = []
# sigma_dust_evol = []

In [10]:
#Boundary Conditions

f_in = 0
f_out = 0

In [11]:
@numba.njit
def make_alpha_grid(cont):
    if(cont):
        alphas = np.linspace(alpha1, alpha3, n)
        return alphas
    else:
        alphas = np.empty(n)
        for i in range(n):
            if i < (n//3):
                alphas[i] = alpha1
            elif i < (2*n//3):
                alphas[i] = alpha2
            else:
                alphas[i] = alpha3
        return alphas

In [12]:
# @numba.njit
def calc_init_params():
    """
    Calculate the initial parameters values for time t = 0 and changing radius.
    """
    #n((dist[i]+dist[i-1])/2)**3
    alphas = make_alpha_grid(cont)
    for i in range(n):
        dist[i] = (rin + (rout-rin)*i/(n-1))
        sigma_gas[i] = sigma_in * AU / dist[i]
        if (sigma_gas[i]>sigma_max):
            sigma_gas[i] = sigma_max
        if (dist[i]/AU > 15):
            sigma_gas[i] = sigma_min
        sigma_dust[i] = sigma_gas[i] * 0.005
        X[i] = 2 * np.sqrt(dist[i])
        Omega[i] = np.sqrt(G * Ms / (dist[i] ** 3))
    cs2, _ , _ = calc_thermal_struc(sigma_gas, sigma_dust, alphas, Omega)
    nu = alphas*cs2/Omega
    D = 12 * nu / (X ** 2)
    f = (1.5 * X * sigma_gas)
    return alphas

In [13]:
def calc_stag_grid(T):
    for i in range(1, n):
        dist_stag[i-1] = 0.5 * (dist[i] + dist[i-1])
        Omega_stag[i-1] = np.sqrt(G * Ms / dist_stag[i-1] ** 3)
#         dPdr[i-1] = (press[i]-press[i-1])/(dist[i] - dist[i-1])
        Tmd = 0.5 * (T[i] + T[i-1])
        cs2_stag[i-1] = (kB * Tmd)/(mu*mH) # we will need the midpoint temperature at some point
#         press_stag[i-1] = 0.5 * (press[i]+press[i-1])
#     ro_stag = press_stag/cs2_stag

In [14]:
def calc_dust_vel(press):
    press_stag = [0.5*(press[i]+press[i-1]) for i in range(1,n)]
    dPdr = [(press[i] - press[i-1])/dist_stag[i-1] for i in range(1,n)]
    rho_stag = press_stag/cs2_stag
    v_dust = v_gas + (St/(1+(St)**2)) * (1 / (rho_stag * Omega_stag))*dPdr
    v_dust = np.append(v_dust, 0)
    return v_dust

In [15]:
# @numba.njit
# def calc_temp():
# #     for i in range(n):
# # #         temp[i] = ((9*sigma[i]*kB*alpha*Omega[i])/(8*sb*mu*mH))**(1/float(3))
# #         temp[i]=100
#     temp = [100 for i in range(n)]
#     return temp

In [16]:
# # @numba.njit
# def calc_press():
#     """Calculate the pressure evolution at each dt at each  r"""
# #     for j in range(n):
# #         cs2[j] = (kB * temp[j])/(mu*mH)
# #         press[j] = np.sqrt(cs2[j])*sigma[j]*Omega[j]
#     press = [np.sqrt((kB * temp[j])/(mu*mH))*sigma_gas[j]*Omega[j] for j in range(n)]
#     return press

In [17]:
@numba.njit
def calc_gas_evol(f, dt):
    """Outputs the surface density at a specific dt."""
    df_dt = np.empty(n)
    for j in range(1, n-1):
        dX1 = X[j] - X[j-1]
        dX2 = X[j+1] - X[j]
        D1 = 0.5 * (D[j] + D[j-1])
        D2 = 0.5 * (D[j+1] + D[j])
        df_dt[j] = D1 * ((-(f[j] - f[j-1])/dX1**2)) + D2 * ((f[j+1]-f[j])/dX2**2)
    dX_final = X[-1]-X[-2]
    dX_in = X[1]-X[0]
    df_dt[0] = D[0] * (-(f[0] - f_in)/dX_in**2 + (f[1]-f[0])/dX_in**2)
    df_dt[n-1] = D[n-1] * (-(f[n-1] - f[n-2])/dX_final**2 + (f_out-f[n-1])/dX_final**2)
    f_new = f + df_dt * dt
    return f_new

In [18]:
# @numba.njit
def calc_disk_mass(sigma):
    """Calculate the mass at a specific dt"""
#     sigma_evol = calc_time_evol(f, alpha, dt)
    disk_mass = 0
    for j in range(n):
        disk_mass += 2 * np.pi * dist[j] * dr * sigma_gas[j]
        disk_mass += 2 * np.pi * dist[j] * dr * sigma_dust[j]
#     print(f'The mass of the disk is {disk_mass}.')
    return disk_mass

In [19]:
@numba.njit
def convert_f2sigma(f_new):
    sigma_at_time = [2*f_new[k]/(3*X[k]) for k in range(n)]
    return sigma_at_time

In [20]:
def save2dir(**alpha_run):
    if(alpha_run):
        if(alpha_run == alpha1):
            output_dir = 'output_a1/'
            filename = 'output_a1/disk_'
        elif(alpha_run == alpha2):
            output_dir = 'output_a2/'
            filename = 'output_a2/disk_'
        else:
            output_dir = 'output_a3/'
            filename = 'output_a3/disk_'
    else:
        output_dir = 'output/'
        filename = 'output/disk_'
    return (output_dir, filename)

In [21]:
#Time Evolution Main

%matplotlib
alphas = calc_init_params()
output_dir, filename = save2dir()
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i in range(0, len(t)):
    f_new = calc_gas_evol(f, dt)
    _, T, P = calc_thermal_struc(sigma_gas, sigma_dust, alphas, Omega)
    calc_stag_grid(T)
    V_dust = calc_dust_vel(P)
    sigma_dust = calc_dust_evol(sigma_gas, sigma_dust, nu, V_dust, dist, dt)
    sigma_gas = convert_f2sigma(f_new)
    f = f_new
    if (i%(t_save_interval/dyr) == 0):
        output = np.vstack([dist/AU, sigma_gas, sigma_dust, V_dust, T, P])
        output = np.transpose(output)
        np.savetxt(filename + str(int(i*dyr)) + '.txt', output, delimiter=',', newline='\n')# if you want to save results to file

Using matplotlib backend: MacOSX


/Users/Rohan/git/surf_2021/dust_diffuse.py:21: RuntimeWarning: invalid value encountered in double_scalars
  Ad[i] = (-dt2/(dr*dr) * (0.5*nds(nu,dist,sigma,i-1) + nds(nu,dist,sigma,i) +0.5*nds(nu,dist,sigma,i+1))/sigma[i]) / dist[i]
/Users/Rohan/git/surf_2021/dust_diffuse.py:23: RuntimeWarning: invalid value encountered in double_scalars
  Ad[i] = (-dt2/(dr*dr) * (0.5*nds(nu,dist,sigma,i-1) + nds(nu,dist,sigma,i))/sigma[i]) / dist[i]
/Users/Rohan/git/surf_2021/dust_diffuse.py:35: RuntimeWarning: invalid value encountered in double_scalars
  Bd[i] =  (dt2/(dr*dr) * 0.5*(nds(nu,dist,sigma,i) + nds(nu,dist,sigma,i+1))/sigma[i+1]) / dist[i]
/Users/Rohan/git/surf_2021/dust_diffuse.py:43: RuntimeWarning: invalid value encountered in double_scalars
  Cd[i] =  dt2/(dr*dr) * 0.5*(nds(nu,dist,sigma,i-1) + nds(nu,dist,sigma,i))/ (sigma[i-1]*dist[i])


KeyboardInterrupt: 

In [ ]:
def plot():
    %matplotlib
    t_plot = [0, 1e3, 5e3, 1e4, 5e4, 1e5, 5e5, 1e6]
    for time in t_plot:
        df = pd.read_csv('output/disk_' + str(time) + '.txt', delimiter=',', header=None)
        sigma_gas_plot = df[1].to_numpy().astype(np.int)
        sigma_dust_plot = df[2].to_numpy().astype(np.int)
        plt.title(f'Surface Density Evolution at t={max_years} years')
        plt.xlabel('Distance (AU)')
        plt.ylabel('Surface Density')
        plt.yscale('log')
        plt.plot(dist/AU, sigma_dust_plot, label='dust')
        plt.plot(dist/AU, sigma_gas_plot, label='gas')
        plt.show()

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [ ]:
# @numba.njit
def compare_code(f):
    """Compare C++ code to Python code"""
    telapsed = [100, 1000]
    count = 0
    for time in telapsed:
        df = pd.read_csv('case/radial_' + str(time) + '.txt', delimiter='\t', header=None)
        sigma_c = df[1].to_numpy().astype(np.int)
        sigma_py = sigma_evol[int((time/t_save_interval)),:]
        plt.title(f'Python vs C++ Surface Density at t={time} years')
        plt.xlabel('Radius (AU)')
        plt.ylabel('Surface Density')
        plt.yscale('log')
        plt.plot(dist/AU, sigma_py, label='Python')
        plt.plot(dist/AU, sigma_c, label='C++')
        plt.legend()
        plt.show()

In [ ]:
# @numba.njit
def plot_rms():
    """Plot RMS of Python vs C++"""
    t_elapsed = [100, 1000]
    t_rms = np.arange(0, final_time, n)
    rms = []
    for time in t_elapsed:
        df = pd.read_csv('case/radial_' + str(time) + '.txt', delimiter='\t', header=None)
        print(df)
        sigma_c = df[1].to_numpy().astype(np.int)
        sigma_py = sigma_evol[int((time/t_save_interval)),:]
        rms.append(np.sum(((sigma_c - sigma_py)/(sigma_c))**2))
    plt.title('Root Mean Surface Density at t={time} years')
    plt.xlabel('Time (years)')
    plt.ylabel('RMS')
    plt.yscale('log')
    plt.plot(t_elapsed, rms)
    plt.show()

In [ ]:
#         sigma_evol.append(sigma)
#         temp_evol.append(T)
#         press_evol.append(P)
#         v_dust_evol.append(V_dust)
#         sigma_dust_evol.append(sigma_dust)
#         mass = calc_disk_mass(sigma)
#         mass_evol.append(mass)
#         t_plot.append(i*dyr)
# sigma_evol = np.array(sigma_evol)
# temp_evol = np.array(temp_evol)
# press_evol = np.array(press_evol)
# v_dust_evol = np.array(v_dust_evol)
# sigma_dust_evol = np.array(sigma_dust_evol)
# for j in range(len(t_plot)):
#     plt.figure(1)
#     plt.title(f'Surface Dust Evolution at t={max_years} years')
#     plt.xlabel('Distance (AU)')
#     plt.ylabel('Sigma Dust')
#     plt.yscale('log')
#     plt.plot(dist/AU, sigma_dust_evol[j,:])
#     plt.show()
#     plt.figure(2)
#     plt.title(f'Surface Gas Evolution at t={max_years} years')
#     plt.xlabel('Distance (AU)')
#     plt.ylabel('Sigma Gas')
#     plt.yscale('log')
#     plt.plot(dist/AU, sigma_evol[j,:])
#     plt.show()




# print(f'First alpha change is at {dist[n//3]/AU}')
# print(f'Second alpha change is at {dist[2*n//3]/AU}')